In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import math
import os
import sys
import warnings
import argparse
import numpy as np
import yaml
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from tqdm import tqdm
import torch
from torch import optim
from torch import Tensor
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset,Sampler
from torchvision import datasets,transforms
import torchvision.transforms as transforms
from torch.nn import functional as F

sys.path.append("/home/shiyi/gpu/gpu/")
sys.path.append("/home/shiyi/colposcopy/colposcopy/util")
img_dir = "/data/colposcopy/colposcopy"
img_dir_target = "/data/colposcopy/colposcopy_1/"

from gpu_allocation import set_gpu

import csv

error = glob(os.path.join(img_dir,"train*","*","2845.jpg"))+ \
    glob(os.path.join(img_dir,"train*","*","5893.jpg"))+glob(os.path.join(img_dir,"train*","*","5892.jpg"))
print(error)

from gpu_allocation import set_gpu
corrupt_set  = set()
types = [1,2,3]

type_1_train = [i for i in glob(os.path.join(img_dir,"train*","Type_1","*.jpg")) if i not in error]
type_2_train = [i for i in glob(os.path.join(img_dir,"train*","Type_2","*.jpg")) if i not in error]
type_3_train = [i for i in glob(os.path.join(img_dir,"train*","Type_3","*.jpg")) if i not in error]

img_test   = glob(os.path.join(img_dir,"test","*.jpg"))
img_test_wo_answer = glob(os.path.join(img_dir,"test_stg2","*.jpg"))

img_paths = type_1_train+type_2_train+type_3_train+img_test+img_test_wo_answer
print(len(img_paths))

[u'/data/colposcopy/colposcopy/train_stg2/Type_2/2845.jpg', u'/data/colposcopy/colposcopy/train_stg2/Type_1/5893.jpg', u'/data/colposcopy/colposcopy/train_stg2/Type_2/5892.jpg']
12230


In [2]:
class ColpoProcessDataset(Dataset):
    def __init__(self):
        pass
    def __getitem__(self, index):
        path = img_paths[index]
        x = cv2.imread(path)
        if x is None:
            return  None,None
        return x
    def __len__(self):
        return len(img_paths)

def data_label_alter(csv_path):
    def data_label_del(target_img_path):
        for i in img_paths:
            if i.split('/')[-1]==target_img_path:
                img_paths.remove(i)
                #print ("%s removed"%i)
                return
                
    def data_label_exchange(target_img_path,src,target):
        for i in range(len(img_paths)):
            path_token = img_paths[i].split('/')
            if path_token[-1]==target_img_path and src == path_token[-2]:
                #print (target_img_path,src,target)
                #print (path_token[-2],src,target)
                path_token[-2] = target
               #print ("From",img_paths[i])
                img_paths[i] = '/'.join(path_token)
               # print ("To",img_paths[i])
                return 
                
        
    with open(csv_path) as f:
        f_csv = csv.reader(f)
        index = 0
        for row in f_csv:
            if index == 0:
                index += 1
                continue
            if row[-1] == 'remove':
                data_label_del(row[0])
            else:
                data_label_exchange(row[0],row[1],row[2])

            
def cvt(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def preprocess(img): 
    r_c = img[:,:,2]
    g_c = img[:,:,1]
    b_c = img[:,:,0]
    _,r_thres = cv2.threshold(r_c, 210, 255, cv2.THRESH_BINARY)
    _,g_thres = cv2.threshold(g_c, 208, 255, cv2.THRESH_BINARY)
    _,b_thres = cv2.threshold(b_c, 206, 255, cv2.THRESH_BINARY)
    mixed = r_thres & b_thres & g_thres
    kernel_dilate = np.ones((18, 18), np.uint8)
    mask = cv2.dilate(mixed,kernel_dilate)
    dst = cv2.inpaint(img,mask,14,cv2.INPAINT_TELEA)
    return cvt(dst)

def collate_fn1(batch):
    x = [item[0] for item in batch if item[0] is not None]
    y = [item[1] for item in batch if item[0] is not None]
    return x,y

In [3]:
print(img_paths[0].split('/')[4:])
data_label_alter(os.path.join(img_dir,"fixed_labels_v2.csv"))
data_label_alter(os.path.join(img_dir,"removed_files.csv"))

[u'train', u'Type_1', u'732.jpg']


In [13]:
print(len(img_paths))

12153


In [5]:
batch_size_train = 16
colpoProcessDataset = ColpoProcessDataset()
process_loader = DataLoader(dataset=colpoProcessDataset,batch_size=batch_size_train,shuffle=True,num_workers = 2)
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
#num_gpu = 1
#set_gpu(num_gpu)
    
for img_path in tqdm(img_paths):
    img_path_target = os.path.join(img_dir_target + '/'.join(img_path.split('/')[4:]))
   # print(img_path_target)
    img = cv2.imread(img_path)
    img = preprocess(img)
    cv2.imwrite(img_path_target,img,[int(cv2.IMWRITE_JPEG_QUALITY),100])

  0%|          | 34/12153 [06:53<32:30:29,  9.66s/it]

TypeError: 'NoneType' object has no attribute '__getitem__'

In [ ]:
import csv
csv_path = (img_dir +'/'+ "removed_files.csv")
f = open(csv_path)
f_csv = csv.reader(f)
for row in f_csv:
    print(row)

In [ ]:
def maxHist(hist):
    maxArea = (0, 0, 0)
    height = []
    position = []
    for i in range(len(hist)):
        if (len(height) == 0):
            if (hist[i] > 0):
                height.append(hist[i])
                position.append(i)
        else: 
            if (hist[i] > height[-1]):
                height.append(hist[i])
                position.append(i)
            elif (hist[i] < height[-1]):
                while (height[-1] > hist[i]):
                    maxHeight = height.pop()
                    area = maxHeight * (i-position[-1])
                    if (area > maxArea[0]):
                        maxArea = (area, position[-1], i)
                    last_position = position.pop()
                    if (len(height) == 0):
                        break
                position.append(last_position)
                if (len(height) == 0):
                    height.append(hist[i])
                elif(height[-1] < hist[i]):
                    height.append(hist[i])
                else:
                    position.pop()    
    while (len(height) > 0):
        maxHeight = height.pop()
        last_position = position.pop()
        area =  maxHeight * (len(hist) - last_position)
        if (area > maxArea[0]):
            maxArea = (area, len(hist), last_position)
    return maxArea
            

def maxRect(img):
    maxArea = (0, 0, 0)
    addMat = np.zeros(img.shape)
    for r in range(img.shape[0]):
        if r == 0:
            addMat[r] = img[r]
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
        else:
            addMat[r] = img[r] + addMat[r-1]
            addMat[r][img[r] == 0] *= 0
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
    return (int(maxArea[3]+1-maxArea[0]/abs(maxArea[1]-maxArea[2])), maxArea[2], maxArea[3], maxArea[1], maxArea[0])

def cropCircle(img):
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    img = cv2.resize(img, dsize=tile_size)
            
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    _, contours, _ = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    main_contour = sorted(contours, key = cv2.contourArea, reverse = True)[0]
            
    ff = np.zeros((gray.shape[0],gray.shape[1]), 'uint8') 
    cv2.drawContours(ff, main_contour, -1, 1, 15)
    ff_mask = np.zeros((gray.shape[0]+2,gray.shape[1]+2), 'uint8')
    cv2.floodFill(ff, ff_mask, (int(gray.shape[1]/2), int(gray.shape[0]/2)), 1)
    
    rect = maxRect(ff)
    rectangle = [min(rect[0],rect[2]), max(rect[0],rect[2]), min(rect[1],rect[3]), max(rect[1],rect[3])]
    img_crop = img[rectangle[0]:rectangle[1], rectangle[2]:rectangle[3]]
    cv2.rectangle(ff,(min(rect[1],rect[3]),min(rect[0],rect[2])),(max(rect[1],rect[3]),max(rect[0],rect[2])),3,2)
    
    return [img_crop, rectangle, tile_size]

def Ra_space(img, Ra_ratio, a_threshold):
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = np.zeros((w*h, 2))
    for i in range(w):
        for j in range(h):
            R = math.sqrt((w/2-i)*(w/2-i) + (h/2-j)*(h/2-j))
            Ra[i*h+j, 0] = R
            Ra[i*h+j, 1] = min(imgLab[i][j][1], a_threshold)
            
    Ra[:,0] /= max(Ra[:,0])
    Ra[:,0] *= Ra_ratio
    Ra[:,1] /= max(Ra[:,1])

    return Ra

def get_and_crop_image(image_id, image_type):
    img = get_image_data(image_id, image_type)
    initial_shape = img.shape
    [img, rectangle_cropCircle, tile_size] = cropCircle(img)
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = Ra_space(imgLab, 1.0, 150)
    a_channel = np.reshape(Ra[:,1], (w,h))
    
    g = mixture.GaussianMixture(n_components = 2, covariance_type = 'diag', random_state = 0, init_params = 'kmeans')
    image_array_sample = shuffle(Ra, random_state=0)[:1000]
    g.fit(image_array_sample)
    labels = g.predict(Ra)
    labels += 1 # Add 1 to avoid labeling as 0 since regionprops ignores the 0-label.
    
    # The cluster that has the highest a-mean is selected.
    labels_2D = np.reshape(labels, (w,h))
    gg_labels_regions = measure.regionprops(labels_2D, intensity_image = a_channel)
    gg_intensity = [prop.mean_intensity for prop in gg_labels_regions]
    cervix_cluster = gg_intensity.index(max(gg_intensity)) + 1

    mask = np.zeros((w * h,1),'uint8')
    mask[labels==cervix_cluster] = 255
    mask_2D = np.reshape(mask, (w,h))

    cc_labels = measure.label(mask_2D, background=0)
    regions = measure.regionprops(cc_labels)
    areas = [prop.area for prop in regions]

    regions_label = [prop.label for prop in regions]
    largestCC_label = regions_label[areas.index(max(areas))]
    mask_largestCC = np.zeros((w,h),'uint8')
    mask_largestCC[cc_labels==largestCC_label] = 255

    img_masked = img.copy()
    img_masked[mask_largestCC==0] = (0,0,0)
    img_masked_gray = cv2.cvtColor(img_masked, cv2.COLOR_RGB2GRAY);
            
    _,thresh_mask = cv2.threshold(img_masked_gray,0,255,0)
            
    kernel = np.ones((11,11), np.uint8)
    thresh_mask = cv2.dilate(thresh_mask, kernel, iterations = 1)
    thresh_mask = cv2.erode(thresh_mask, kernel, iterations = 2)
    _, contours_mask, _ = cv2.findContours(thresh_mask.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    main_contour = sorted(contours_mask, key = cv2.contourArea, reverse = True)[0]
    cv2.drawContours(img, main_contour, -1, 255, 3)
    
    x,y,w,h = cv2.boundingRect(main_contour)
    
    rectangle = [x+rectangle_cropCircle[2],
                 y+rectangle_cropCircle[0],
                 w,
                 h,
                 initial_shape[0],
                 initial_shape[1],
                 tile_size[0],
                 tile_size[1]]

    return [image_id, img, rectangle]